In [1]:
%load_ext autoreload
%autoreload 2
%gui qt
%matplotlib qt5

import sys
import os

import numpy as np
import matplotlib.pyplot as plt

### Load Metadata module

In [2]:
from oyLabImaging import Metadata

### Define path

In [3]:
fpath = "/bigstore/TestSpin/08132021_TrackingCells_mRubyLoss/_1/"
#fpath = "/bigstore/Microscopy Core/Jen/A549Inf_Drugs.nd2"
#fpath="/bigstore/Microscopy Core/Jen/DIC_timelapseDeathComparison_08032021.pickle"
#fpath ='/bigstore/TestSpin/08132021_TrackingCells_mRubyLoss/_1/metadata.pickle'
fpath = '/bigstore/Microscopy Core/Jen/3T3_mRubyloss_HSV_20210818/'
#fpath = '/bigstore/Microscopy Core/Jen'
#fpath = '/bigstore/Microscopy Core/Jen/3T3_mRubyloss_HSV_20210818/MRubyVirusDeath_08172021.nd2'
MD = Metadata(fpath)

loaded ND2 metadata from/bigstore/Microscopy Core/Jen/3T3_mRubyloss_HSV_20210818/metadata.pickle


### You can read images by attributes

In [4]:
stk = MD.stkread(Channel = MD.channels[1], Position=MD.posnames[1], Zindex=[0],register=True)

opening index 8206
Loaded 1 group of images.


In [5]:
MD.CalculateDriftCorrection(Channel='DeepBlue')

opening file img_channel000_position002_time000000059_z000.tif
Loaded Pos2 group of images.

calculating drift correction for position Pos2
calculated drift correction for position Pos2
opening file img_channel000_position000_time000000059_z000.tif
Loaded Pos0 group of images.

calculating drift correction for position Pos0
calculated drift correction for position Pos0
opening file img_channel000_position001_time000000059_z000.tif
Loaded Pos1 group of images.

calculating drift correction for position Pos1
calculated drift correction for position Pos1


### use napari to see images

In [5]:
import napari
viewer = napari.Viewer()

2021-09-09 15:59:52,960 [INFO] WRITING LOG OUTPUT TO /home/alo649/.cellpose/run.log


In [6]:
viewer.add_image(stk)

<Image layer 'stk' at 0x7f79282f7760>

### If you want to segment cells, you can use a function called test_segmentation_params to see how the algorythm performs with different parameters. Any parameter value that's changed from the default should be manually added to the call for segmentation of the full data below.

In [9]:
from oyLabImaging.Processing.improcutils import segmentation
print(segmentation.segmentation_types()) 
img = MD.stkread(Position=MD.posnames[1],Channel=MD.channels[0], frame=10)
segmentation.test_segmentation_params(img=img, segment_type='cellpose_nuclei', cellprob_threshold=0.0)#cellpose_nuclei

['watershed', 'cellpose_nuclei', 'cellpose_cyto', 'cellpose_nuc_cyto']
opening file img_channel000_position000_time000000010_z000.tif
Loaded Pos0 group of images.

using _segment_nuclei_cellpose


calculating with new parameters
Done!


### make empty results object

In [5]:
from oyLabImaging.Processing import results
R = results(MD=MD)


loaded results from pickle file


In [6]:
R()

Results object for path to experiment in path: 
 /bigstore/Microscopy Core/Jen/3T3_mRubyloss_HSV_20210818/

Available channels are : DIC N2, Widefield Green, Widefield Red, Widefield Dual Cyan, Widefield Blue.

Positions already segmented are : 0

Available positions : 0, 1, 2, 3, 4, 5.

Available frames : 92.


In [7]:
R.setPosLbls(MD=MD,Position=R.PosNames[0],NucChannel=R.channels[0],segment_type='cellpose_nuclei', threads=6, diameter=50)


Processing position 0


100%|███████████████████████████████████████████| 92/92 [00:42<00:00,  2.15it/s]



Finished loading and segmenting position 0
saved results


In [11]:
MD.channels

array(['DeepBlue', 'Green', 'Red'], dtype=object)

In [8]:
R.calculate_tracks(pos=R.PosNames[0], NucChannel=MD.channels[0], search_radius=40, maxStep=30)

linking frame 58
Finished connecting tracks
saved results


In [9]:
R.show_points(R.PosNames[0])

showing channel DeepBlue
2021-09-09 15:48:47,663 [INFO] WRITING LOG OUTPUT TO /home/alo649/.cellpose/run.log


In [20]:
R.show_tracks(R.PosNames[0], J=np.arange(50))

showing channel DeepBlue


In [35]:
t0 = R.tracks(0)

In [37]:
i=20
plt.plot(t0(i).T, t0(i).mean(MD.channels[4]))
plt.show()